In [1]:
from ogb.graphproppred import GraphPropPredDataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score
import pickle
import ogbcomp_helper as helper

# Run classification

In [2]:
all_probs = {}
all_ap = {}
all_rocs = {}
train_label_props = {}

In [6]:
quick_test = False

In [7]:
if quick_test:
    n_estimators = 100
    max_tasks = 3
    nreps = 1
else:
    n_estimators = 1000
    max_tasks = None
    nreps = 4

In [12]:
for rep in range(nreps):
    for ds in ["ogbg-molpcba", "ogbg-molhiv"]:
        # Read in dataset
        print("Reading dataset")
        dataset = GraphPropPredDataset(name=ds)
        
        # By default, the line above will save data to the path below
        df_smi = pd.read_csv(f"dataset/{ds}/mapping/mol.csv.gz".replace("-", "_"))
        smiles = df_smi["smiles"]
        outcomes = df_smi.set_index("smiles").drop(["mol_id"], axis=1)
        
        # Make sure smiles strings and graphs are in the same order so splits are valid
        helper.spotcheck_order(smiles, dataset)

        # Generate features
        print("Extracting features...")
        X = helper.parallel_apply(smiles, func=helper.getmorganfingerprint2)

        # Split into train/val/test
        split_idx = dataset.get_idx_split()
        train_idx, val_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
        X_train, X_val, X_test = X.iloc[train_idx], X.iloc[val_idx], X.iloc[test_idx]

        for oo in tqdm(outcomes.columns[:max_tasks]):
            # Get probabilities
            val_key = ds, oo, rep, "val"
            test_key = ds, oo, rep, "test"
            
            # If re-running, skip finished runs
            if val_key in all_probs:
                print("Skipping", val_key[:-1])
                continue
    
            # Split outcome in to train/val/test
            Y = outcomes[oo]
            y_train, y_val, y_test = Y.loc[X_train.index], Y.loc[X_val.index], Y.loc[X_test.index]
            
            # Skip outcomes with no positive training examples
            if y_train.sum() == 0:
                continue
            
            # Remove missing labels in validation
            y_val, y_test = y_val.dropna(), y_test.dropna()
            X_v, X_t = X_val.loc[y_val.index], X_test.loc[y_test.index]
            
            # Remove missing values in the training labels, and downsample imbalance to cut runtime
            y_tr = helper.process_y(y_train)
            train_label_props[ds, oo, rep] = y_tr.mean()
            print(f"Sampled label balance:\n{y_tr.value_counts()}")
            
            # Fit model
            print("Fitting model...")
            rf = RandomForestClassifier(min_samples_leaf=2, n_estimators=n_estimators, n_jobs=-1)
            rf.fit(X_train.loc[y_tr.index], y_tr)

            # Calculate probabilities
            all_probs[val_key] = pd.Series(rf.predict_proba(X_v)[:, 1], index=X_v.index)
            all_probs[test_key] = pd.Series(rf.predict_proba(X_t)[:, 1], index=X_t.index)

            if y_val.sum() > 0:
                all_ap[val_key] = average_precision_score(y_val, all_probs[val_key])
                all_rocs[val_key] = roc_auc_score(y_val, all_probs[val_key])
            
            if y_test.sum() > 0:
                all_ap[test_key] = average_precision_score(y_test, all_probs[test_key])
                all_rocs[test_key] = roc_auc_score(y_test, all_probs[test_key])

            print(f'{oo}, rep {rep}, AP (val, test): {all_ap.get(val_key, np.nan):.3f}, {all_ap.get(test_key, np.nan):.3f}')
            print(f'\tROC (val, test): {all_rocs.get(val_key, np.nan):.3f}, {all_rocs.get(test_key, np.nan):.3f}')

Reading dataset


  2%|▏         | 79/5000 [00:00<00:06, 781.36it/s]

Spot checking order between mapping file and graphs...


100%|██████████| 5000/5000 [00:04<00:00, 1020.43it/s]

Extracting features...


...

Catenating results


  0%|          | 0/1 [00:00<?, ?it/s]

Sampled label balance:
0    31669
1     1232
Name: HIV_active, dtype: int64
Fitting model...


100%|██████████| 1/1 [00:06<00:00,  6.06s/it]

HIV_active, rep 0, AP (val, test): 0.386, 0.408
	ROC (val, test): 0.833, 0.798


# Save results

In [19]:
os.mkdir("ogb_results")

In [17]:
with open("ogb_results/results.pkl", "wb") as fout:
    all_results = {
        "probs": all_probs,
        "AP": all_ap,
        "ROCs": all_rocs,
        "Label_props": train_label_props
    }
    pickle.dump(all_results, fout)

# Read results from disk

In [7]:
with open("ogb_results/results.pkl", "rb") as fin:
    all_results = pickle.load(fin)

In [8]:
all_probs = all_results["probs"]

# Evaluate predictions

In [9]:
for task in ["ogbg-molpcba", "ogbg-molhiv"]:
    outcomes = (
        pd.read_csv(f"dataset/{task}/mapping/mol.csv.gz".replace("-", "_"))
        .set_index("smiles")
        .drop(["mol_id"], axis=1)
    )
    
    evaluator = Evaluator(name=task)
    
    print(f"\n=== {task} ===")
    for dset in ["val", "test"]:
        scores = []
        probs = helper.probs_dict2df(all_probs, task=task, dset=dset)
        for rep in range(4):
            tt = probs[rep]
            input_dict = {"y_true": outcomes.loc[tt.index, tt.columns].values, 
                          "y_pred": tt.values}
            scores.append(evaluator.eval(input_dict))
        print(f"--> {dset}")
        print(pd.DataFrame(scores).agg([np.mean, np.std]))
        print()


=== ogbg-molpcba ===
--> val
            ap
mean  0.222616
std   0.000232

--> test
            ap
mean  0.205363
std   0.000363


=== ogbg-molhiv ===
--> val
        rocauc
mean  0.842090
std   0.003541

--> test
        rocauc
mean  0.806663
std   0.000983

